In [ ]:
import pandas as pd
import numpy as np
import sklearn

print(pd.__version__)
print(np.__version__)
print(sklearn.__version__)

In [ ]:
import sys
from pathlib import Path

PROJECT_ROOT = Path().resolve().parents[0]
sys.path.append(str(PROJECT_ROOT))
print(PROJECT_ROOT)

In [ ]:
from src.data_loader import load_data
import src.data_loader as dl
print(dl.__file__)

In [ ]:
train_data = load_data('../data/raw/train.csv')
test_data = load_data('../data/raw/test.csv')

In [ ]:
from src.preprocessing import fill_age, compute_missing_params, preprocess_data

In [ ]:
from src.features import get_features

In [ ]:
from src.models import train_lr, train_rf

In [ ]:
params = compute_missing_params(train_data)

X_train, X_test = preprocess_data(train_data, test_data, params)
passenger_ids = X_test["PassengerId"]
X_train = get_features(X_train)
X_test  = get_features(X_test)
y_train = train_data["Survived"].astype(int)

print(X_train.shape)
print(X_test.shape)
print(X_train.isnull().sum().sum())

In [ ]:
rf_model = train_rf(X_train, y_train, n_estimators=300, min_samples_leaf=5)

In [ ]:
from src.submit import make_submission

In [ ]:
make_submission(rf_model, X_test, passenger_ids, "./submissions/submission.csv")